In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c alpha-dent

100% 4.55G/4.56G [01:00<00:00, 123MB/s]
100% 4.56G/4.56G [01:00<00:00, 80.4MB/s]


In [ ]:
!unzip alpha-dent.zip -d ./alpha_dent_data

Archive:  alpha-dent.zip
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_000.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_001.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_002.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_003.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_004.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_005.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_006.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_007.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_008.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_009.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_010.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_011.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_012.jpg  
  inflating: ./alpha_dent_data/AlphaDent/images/test/test_013.jpg  
  inflating: ./alpha_de

---

In [ ]:
!pip install -q ultralytics

In [1]:
import os
import sys
import time
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm
import yaml
import random
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

os.environ['WANDB_DISABLED'] = 'true'

random.seed(42)
np.random.seed(42)

os.system('pip install -q ultralytics')

import torch
from ultralytics import YOLO

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [2]:
print(f'\nPyTorch Version: {torch.__version__}')
print(f'CUDA Available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA Device: {torch.cuda.get_device_name(0)}')


PyTorch Version: 2.6.0+cu124
CUDA Available: True
CUDA Device: Tesla T4


In [3]:
BASE_PATH = '/content/alpha_dent_data/AlphaDent'
TRAIN_IMAGES_PATH = f'{BASE_PATH}/images/train'
VALID_IMAGES_PATH = f'{BASE_PATH}/images/valid'
TEST_IMAGES_PATH = f'{BASE_PATH}/images/test'
TRAIN_LABELS_PATH = f'{BASE_PATH}/labels/train'
VALID_LABELS_PATH = f'{BASE_PATH}/labels/valid'

OUTPUT_DIR = '/content/'
WEIGHTS_DIR = f'{OUTPUT_DIR}/weights'
os.makedirs(WEIGHTS_DIR, exist_ok=True)

In [4]:
CLASS_INFO = {
    0: {'name': 'Abrasion', 'description': 'Teeth with mechanical wear of hard tissues'},
    1: {'name': 'Filling', 'description': 'Dental fillings of various types'},
    2: {'name': 'Crown', 'description': 'Dental crown (restoration)'},
    3: {'name': 'Caries Class 1', 'description': 'Caries in fissures and pits'},
    4: {'name': 'Caries Class 2', 'description': 'Caries on proximal surfaces of molars/premolars'},
    5: {'name': 'Caries Class 3', 'description': 'Caries on proximal surfaces of incisors/canines without incisal edge'},
    6: {'name': 'Caries Class 4', 'description': 'Caries on proximal surfaces of incisors/canines with incisal edge'},
    7: {'name': 'Caries Class 5', 'description': 'Cervical caries (buccal/lingual surfaces)'},
    8: {'name': 'Caries Class 6', 'description': 'Caries on incisal edges or cusps'}
}

In [5]:
yolo_config = {
    'path': BASE_PATH,
    'train': 'images/train',
    'val': 'images/valid',
    'test': 'images/test',
    'nc': 9,
    'names': [CLASS_INFO[i]['name'] for i in range(9)]
}

In [6]:
CUSTOM_YAML_PATH = f'{OUTPUT_DIR}/alphadent_config.yaml'
with open(CUSTOM_YAML_PATH, 'w') as f:
    yaml.dump(yolo_config, f, default_flow_style=False)
print(f"Created custom YAML config at: {CUSTOM_YAML_PATH}")

Created custom YAML config at: /content//alphadent_config.yaml


In [7]:
train_images = sorted(glob.glob(f'{TRAIN_IMAGES_PATH}/*.jpg'))
valid_images = sorted(glob.glob(f'{VALID_IMAGES_PATH}/*.jpg'))
test_images = sorted(glob.glob(f'{TEST_IMAGES_PATH}/*.jpg'))

In [8]:
def analyze_class_distribution(labels_path):
    """Analyze class distribution in dataset."""
    class_counts = {i: 0 for i in range(9)}
    total_annotations = 0

    label_files = glob.glob(f'{labels_path}/*.txt')

    for label_file in tqdm(label_files, desc="Analyzing labels", leave=False):
        if os.path.exists(label_file) and os.path.getsize(label_file) > 0:
            try:
                with open(label_file, 'r') as f:
                    lines = f.readlines()
                    for line in lines:
                        if line.strip():
                            parts = line.strip().split()
                            if parts:
                                class_id = int(parts[0])
                                if 0 <= class_id < 9:
                                    class_counts[class_id] += 1
                                    total_annotations += 1
            except Exception:
                continue

    return class_counts, total_annotations

In [9]:
train_class_counts, train_total = analyze_class_distribution(TRAIN_LABELS_PATH)
valid_class_counts, valid_total = analyze_class_distribution(VALID_LABELS_PATH)

print(f"Training set: {train_total} total annotations")
print(f"Validation set: {valid_total} total annotations")

Analyzing labels:   0%|          | 0/1237 [00:00<?, ?it/s]

Analyzing labels:   0%|          | 0/83 [00:00<?, ?it/s]

Training set: 12032 total annotations
Validation set: 872 total annotations


In [10]:
EPOCHS = 30
IMAGE_SIZE = 512
BATCH_SIZE = 4 if torch.cuda.is_available() else 2
PATIENCE = 5

model = YOLO('yolov8s-seg.pt')

results = model.train(
    data=CUSTOM_YAML_PATH,
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    save=True,
    save_period=10,
    project=OUTPUT_DIR,
    name='alphadent_yolov8s',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.0,
    plots=True,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=2,
    verbose=True,
    amp=True,
    val=True
)

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content//alphadent_config.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=alphadent_yolov8s, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12

train: Scanning /content/alpha_dent_data/AlphaDent/labels/train.cache... 1237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1237/1237 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2095.7±1685.0 MB/s, size: 3172.8 KB)


val: Scanning /content/alpha_dent_data/AlphaDent/labels/valid.cache... 83 images, 0 backgrounds, 0 corrupt: 100%|██████████| 83/83 [00:00<?, ?it/s]


Plotting labels to /content/alphadent_yolov8s/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 2 dataloader workers
Logging results to /content/alphadent_yolov8s
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  18%|█▊        | 2/11 [00:00<00:03,  2.93it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:09<00:00,  1.14it/s]


                   all         83        872      0.577      0.342      0.364      0.232      0.567       0.34      0.351      0.198
              Abrasion         73        409      0.615      0.893      0.836      0.744      0.614      0.895      0.833      0.647
               Filling         48        186      0.639      0.613      0.608       0.32      0.619      0.602      0.594      0.273
                 Crown          9         19      0.414      0.632      0.702      0.578      0.417       0.64      0.702      0.513
        Caries Class 1         30         62      0.382      0.339      0.335      0.138      0.346      0.323      0.267     0.0926
        Caries Class 2         41         73       0.52      0.151      0.266     0.0936      0.513      0.151       0.25     0.0679
        Caries Class 3         23         33      0.103     0.0303     0.0815     0.0176     0.0878     0.0303     0.0728     0.0157
        Caries Class 4          3          4          1          0   

In [11]:
best_model_path = f'{OUTPUT_DIR}/alphadent_yolov8s/weights/best.pt'

In [12]:
try:
    metrics = model.val(
        data=CUSTOM_YAML_PATH,
        imgsz=IMAGE_SIZE,
        batch=1,
        conf=0.001,
        iou=0.5,
        max_det=300,
        device=0 if torch.cuda.is_available() else 'cpu',
        plots=False,
        save_json=False,
    )

    print(f"\nValidation Results:")
    print(f"mAP@50: {metrics.seg.map50:.4f}")
    print(f"mAP@50-95: {metrics.seg.map:.4f}")
except Exception as e:
    print(f"Validation error (non-critical): {e}")


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,783,083 parameters, 0 gradients, 42.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3673.7±462.5 MB/s, size: 3129.0 KB)


val: Scanning /content/alpha_dent_data/AlphaDent/labels/valid.cache... 83 images, 0 backgrounds, 0 corrupt: 100%|██████████| 83/83 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 83/83 [00:11<00:00,  6.99it/s]

                   all         83        872      0.548      0.368      0.374      0.231      0.562       0.36      0.362      0.197
              Abrasion         73        409      0.573      0.916      0.836      0.747      0.582       0.91      0.834      0.644
               Filling         48        186      0.628      0.608      0.617      0.314      0.656      0.602      0.609      0.264
                 Crown          9         19      0.393      0.684      0.701      0.566      0.408      0.684      0.701      0.501
        Caries Class 1         30         62      0.366      0.387      0.375      0.148      0.397      0.351      0.282     0.0979
        Caries Class 2         41         73      0.346      0.192      0.285      0.096      0.379      0.184      0.269     0.0686
        Caries Class 3         23         33      0.189     0.0909     0.0879     0.0114      0.134     0.0606     0.0826      0.016
        Caries Class 4          3          4          1          0   


Validation Results:
mAP@50: 0.3616
mAP@50-95: 0.1970


In [ ]:
def convert_to_submission_format(results, image_paths):
    submission_rows = []

    for idx, result in enumerate(results):
        # image ID
        image_id = os.path.basename(image_paths[idx]).replace('.jpg', '')

        if result.masks is not None and len(result.masks) > 0:
            try:
                # Get masks, classes, and confidences
                masks = result.masks.xy
                classes = result.boxes.cls.cpu().numpy().astype(int)
                confidences = result.boxes.conf.cpu().numpy()
                h, w = result.orig_shape

                # Process each detection
                for mask_idx in range(len(masks)):
                    if mask_idx < len(classes) and mask_idx < len(confidences):
                        polygon = masks[mask_idx]

                        if len(polygon) >= 3:  # Valid polygon (at least 3 points)
                            # Normalize coordinates to [0, 1]
                            normalized_coords = []
                            for point in polygon:
                                x_norm = float(point[0]) / w
                                y_norm = float(point[1]) / h
                                # Ensure coordinates are within [0, 1]
                                x_norm = max(0.0, min(1.0, x_norm))
                                y_norm = max(0.0, min(1.0, y_norm))
                                normalized_coords.extend([x_norm, y_norm])

                            # Format polygon string
                            poly_str = ' '.join([f'{coord:.6f}' for coord in normalized_coords])

                            submission_rows.append({
                                'patient_id': image_id,
                                'class_id': int(classes[mask_idx]),
                                'confidence': float(confidences[mask_idx]),
                                'poly': poly_str
                            })
            except Exception as e:
                print(f"Error processing result for image {idx}: {e}")
                continue

    return submission_rows

In [14]:
test_images = sorted(glob.glob(f'{TEST_IMAGES_PATH}/*.jpg'))
all_submission_rows = []
INFERENCE_BATCH_SIZE = 4 if torch.cuda.is_available() else 2

In [15]:
for i in tqdm(range(0, len(test_images), INFERENCE_BATCH_SIZE)):
    batch_images = test_images[i:i + INFERENCE_BATCH_SIZE]

    try:
        results = model.predict(
            batch_images,
            imgsz=IMAGE_SIZE,
            conf=0.25,  # Confidence threshold
            iou=0.45,   # NMS IoU threshold
            max_det=300,
            device=0 if torch.cuda.is_available() else 'cpu',
            verbose=False,
            agnostic_nms=True,
            retina_masks=True,
        )

        # Convert results to submission format
        batch_rows = convert_to_submission_format(results, batch_images)
        all_submission_rows.extend(batch_rows)

    except Exception as e:
        print(f"Error in batch {i//INFERENCE_BATCH_SIZE}: {e}")
        continue

print(f"\nGenerated {len(all_submission_rows)} predictions")

  0%|          | 0/34 [00:00<?, ?it/s]


Generated 1414 predictions


In [16]:
submission_df = pd.DataFrame(all_submission_rows)

In [17]:
all_test_ids = [os.path.basename(img).replace('.jpg', '') for img in test_images]
if len(submission_df) > 0:
    predicted_ids = submission_df['patient_id'].unique()
    missing_ids = set(all_test_ids) - set(predicted_ids)
else:
    missing_ids = set(all_test_ids)

# Add dummy predictions for images without detections
if missing_ids:
    print(f"Adding dummy predictions for {len(missing_ids)} images without detections")
    dummy_rows = []
    for img_id in missing_ids:
        # Create a small dummy polygon
        dummy_rows.append({
            'patient_id': img_id,
            'class_id': 0,
            'confidence': 0.01,
            'poly': '0.1 0.1 0.1 0.2 0.2 0.2 0.2 0.1'
        })

    submission_df = pd.concat([submission_df, pd.DataFrame(dummy_rows)], ignore_index=True)

Adding dummy predictions for 1 images without detections


In [18]:
submission_df = submission_df.sort_values(['patient_id', 'confidence'], ascending=[True, False])

# correct column order
submission_df = submission_df[['patient_id', 'class_id', 'confidence', 'poly']]

submission_df.to_csv('submission.csv', index=False)

In [19]:
submission_df.head()

,patient_id,class_id,confidence,poly
0,test_000,7,0.785457,0.231250 0.644792 0.230729 0.645573 0.230556 0...
1,test_000,0,0.753609,0.436458 0.350521 0.436458 0.350781 0.436111 0...
2,test_000,7,0.665054,0.353993 0.642448 0.353993 0.669792 0.356597 0...
3,test_000,7,0.636561,0.739236 0.632292 0.739236 0.646094 0.739583 0...
4,test_000,7,0.574530,0.590972 0.604688 0.590972 0.621875 0.591146 0...


In [20]:
missing_in_submission = set(all_test_ids) - set(submission_df['patient_id'].unique())
if missing_in_submission:
    print(f"Missing images in submission: {missing_in_submission}")
else:
    print("test images have predictions")

test images have predictions


In [21]:
print("\nPredictions per class:")
class_dist = submission_df['class_id'].value_counts().sort_index()
for class_id, count in class_dist.items():
    if 0 <= class_id < 9:
        print(f"  Class {class_id} ({CLASS_INFO[class_id]['name']}): {count}")


Predictions per class:
  Class 0 (Abrasion): 797
  Class 1 (Filling): 264
  Class 2 (Crown): 70
  Class 3 (Caries Class 1): 51
  Class 4 (Caries Class 2): 53
  Class 5 (Caries Class 3): 21
  Class 7 (Caries Class 5): 159


In [22]:
high_conf_df = submission_df[submission_df['confidence'] >= 0.3].copy()

high_conf_ids = high_conf_df['patient_id'].unique()
missing_high_conf = set(all_test_ids) - set(high_conf_ids)

if missing_high_conf:
    for img_id in missing_high_conf:
        img_preds = submission_df[submission_df['patient_id'] == img_id]
        if len(img_preds) > 0:
            high_conf_df = pd.concat([high_conf_df, img_preds.head(1)], ignore_index=True)
        else:
            dummy_row = pd.DataFrame([{
                'patient_id': img_id,
                'class_id': 0,
                'confidence': 0.01,
                'poly': '0.1 0.1 0.1 0.2 0.2 0.2 0.2 0.1'
            }])
            high_conf_df = pd.concat([high_conf_df, dummy_row], ignore_index=True)

high_conf_df = high_conf_df.sort_values(['patient_id', 'confidence'], ascending=[True, False])
high_conf_df.to_csv('submission_high_conf.csv', index=False)
print(f"Created high confidence submission with {len(high_conf_df)} predictions")

Created high confidence submission with 1240 predictions
